In [52]:
import pandas as pd
# pd.set_option('display.max_rows', 300)
import rapidfuzz
from joblib import Parallel, delayed
from tqdm import tqdm
import re
import awswrangler as wr
import boto3
import numpy as np

In [53]:

wr.config.s3_endpoint_url = "http://192.168.1.4:8333"

labelDf = wr.s3.read_parquet(
    path=f"s3://partlist_label/",
    columns=["CaseID", "lvl_1_desc", "lvl_2_desc", "lvl_3_desc" ,"Description",	"fVAL",	"DamageCond"],
)
labelDf

,CaseID,lvl_1_desc,lvl_2_desc,lvl_3_desc,Description,fVAL,DamageCond
0,12657878,bonnet,bonnet,bonnet,bonnet,550.00,bent/dented
1,12657888,bonnet,bonnet,bonnet,bonnet,510.44,bent
2,12657890,bonnet,bonnet,bonnet,bonnet,504.94,dented
3,12657900,bonnet,bonnet,bonnet,bonnet,450.00,crumpled
4,12657903,bonnet,bonnet,bonnet,bonnet,280.00,bent
...,...,...,...,...,...,...,...
7235543,12606480,rear panel inner trim,rear panel,rear panel,rear panel inner trim,250.00,broken
7235544,12606779,rear panel inner trim,rear panel,rear panel,firewall rear trim,225.00,
7235545,12607452,rear panel inner trim,rear panel,rear panel,inner carrier rh,384.00,holder torn
7235546,12607489,rear panel inner trim,rear panel,rear panel,taillamp inner lh n rh,691.00,broken


In [72]:
tempDf = labelDf[labelDf["CaseID"] == 10053736]
tempDf

,CaseID,lvl_1_desc,lvl_2_desc,lvl_3_desc,Description,fVAL,DamageCond
135439,10053736,bonnet,bonnet,bonnet,bonnet,282.85,
438808,10053736,bumper front cover,bumper front,bumper front,bumper front cover,222.98,
3864453,10053736,fender front rh,fender front rh,fender front rh,fender front rh,100.80,
1997165,10053736,grille front,grille,grille,grille front,131.90,
1291892,10053736,headlamp assy rh,headlamp rh,headlamp rh,headlamp assy rh,373.74,


In [55]:
caseStudyDf = pd.read_csv("/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/case_study/case_study_4/case_study_gt.csv")
caseStudyCaseId = caseStudyDf["CaseID"].tolist()


In [56]:
labelDf = labelDf[labelDf["CaseID"].isin(caseStudyCaseId)]
labelDf.sort_values(by=["CaseID", "lvl_3_desc"], inplace=True)
labelDf

/tmp/ipykernel_26798/4094135370.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,CaseID,lvl_1_desc,lvl_2_desc,lvl_3_desc,Description,fVAL,DamageCond
135439,10053736,bonnet,bonnet,bonnet,bonnet,282.85,
438808,10053736,bumper front cover,bumper front,bumper front,bumper front cover,222.98,
3864453,10053736,fender front rh,fender front rh,fender front rh,fender front rh,100.80,
1997165,10053736,grille front,grille,grille,grille front,131.90,
1291892,10053736,headlamp assy rh,headlamp rh,headlamp rh,headlamp assy rh,373.74,
...,...,...,...,...,...,...,...
37320,13045105,bonnet,bonnet,bonnet,bonnet,412.10,grazed/bent
290608,13045105,bumper front cover,bumper front,bumper front,bumper front cover,220.00,torn
3801979,13045105,fender front rh,fender front rh,fender front rh,fender front rh,179.70,crumpled
1174571,13045105,headlamp assy rh,headlamp rh,headlamp rh,headlamp assy rh,300.00,broken


In [57]:
outputDir = "/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/case_study/case_study_4/partlist"

In [71]:
import plotly.graph_objects as go
for caseId in labelDf["CaseID"].unique().tolist():
    df = labelDf[labelDf["CaseID"] == caseId]
    fig = go.Figure(data=[go.Table(
        header=dict(values=list(df.columns),                line_color='darkslategray',
),
        cells=dict(values=[df[x] for x in df.columns],                line_color='darkslategray',
)
    )])

    fig.update_layout(width=1200, height=( np.clip(
        
            (len(df) // 4)* 300, 
            a_min=400, a_max=3000
            )
            )  
            )

# fig.show()
    fig.write_image(f"{outputDir}/{caseId}.png") 


In [67]:
multilabelDf = wr.s3.read_parquet(
        path=f"s3://multilabel_df/",
        dataset=True,
    )


In [78]:
allTargetCol = [
 "headlamp_rh",'engine', 'airbag', 'front_panel', 'undercarriage', 'wheel', 'fender_front_lh', 'fender_front_rh', 'door_mirror_rh', 'rear_quarter_lh', 'interior', 'tail_lamp_lh', 'fog_lamp_lh', 'tail_lamp_rh', 'windscreen_front', 'rear_compartment', 'rear_panel', 'rear_quarter_rh', 'door_rear_rh', 'door_mirror_lh', 'door_rear_lh', 'windscreen_rear'
]

In [79]:
temp = multilabelDf[multilabelDf["CaseID"] == 12393479]
temp[allTargetCol]

,headlamp_rh,engine,airbag,front_panel,undercarriage,wheel,fender_front_lh,fender_front_rh,door_mirror_rh,rear_quarter_lh,...,fog_lamp_lh,tail_lamp_rh,windscreen_front,rear_compartment,rear_panel,rear_quarter_rh,door_rear_rh,door_mirror_lh,door_rear_lh,windscreen_rear
248150,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
